# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [1]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [2]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

# Generate with the base model

Here we will try out the base model which does not have a chat template. 

In [5]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

{'input_ids': tensor([[    1,  4093,   198, 19161,   253,   421, 30614,   563,  6256,     2,
           198]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([[    1,  4093,   198, 19161,   253,   421, 30614,   563,  6256,     2,
           198, 19161,   253,   421, 30614,   563,  6256,   198, 19161,   253,
           421, 30614,   563,  6256,   198, 19161,   253,   421, 30614,   563,
          6256,   198, 19161,   253,   421, 30614,   563,  6256,   198, 19161,
           253,   421, 30614,   563,  6256,   198, 19161,   253,   421, 30614,
           563,  6256,   198, 19161,   253,   421, 30614,   563,  6256,   198,
         19161,   253,   421, 30614,   563,  6256,   198, 19161,   253,   421,
         30614,   563,  6256,   198, 19161,   253,   421, 30614,   563,  6256,
           198, 19161,   253,   421, 30614,   563,  6256,   198, 19161,   253,
           421, 30614,   563,  6256,   198, 19161,   253,   421, 30614,   563,
          6256,   198, 19161,   253

## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [11]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [8]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)

In [12]:
print(ds["train"][3])

{'full_topic': 'Cooking/Cooking for others/Food gifting', 'messages': [{'content': 'Hi there', 'role': 'user'}, {'content': 'Hello! How can I help you today?', 'role': 'assistant'}, {'content': "I'm planning to cook dinner for my family tonight, do you have any easy recipe suggestions?", 'role': 'user'}, {'content': 'Yes, I can suggest a simple pasta recipe with tomato sauce and vegetables. Would you like me to share the steps?', 'role': 'assistant'}, {'content': "That sounds great, I'd love to hear the steps. And also, I want to gift some food to my neighbor, what's a good food gift idea?", 'role': 'user'}, {'content': 'For the pasta recipe, just boil pasta, heat canned tomato sauce with sautéed vegetables, and combine. For a food gift, consider baking cookies or making a fruit basket – both are easy and appreciated.', 'role': 'assistant'}]}


Note: you may need to restart the kernel to use updated packages.


d:\ai\agent\workspace\smol-course\.venv\Scripts\python.exe: No module named pip


## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [13]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [20]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

  0%|          | 0/1000 [00:00<?, ?it/s]

{'loss': 0.6056, 'grad_norm': 1.4867925643920898, 'learning_rate': 4.9500000000000004e-05, 'epoch': 0.02}
{'loss': 0.5477, 'grad_norm': 1.507423996925354, 'learning_rate': 4.9e-05, 'epoch': 0.04}
{'loss': 0.5677, 'grad_norm': 1.650034785270691, 'learning_rate': 4.85e-05, 'epoch': 0.05}
{'loss': 0.7256, 'grad_norm': 2.003661632537842, 'learning_rate': 4.8e-05, 'epoch': 0.07}
{'loss': 0.6995, 'grad_norm': 1.894800066947937, 'learning_rate': 4.75e-05, 'epoch': 0.09}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.065290093421936, 'eval_runtime': 31.7472, 'eval_samples_per_second': 3.748, 'eval_steps_per_second': 0.472, 'epoch': 0.09}
{'loss': 0.7676, 'grad_norm': 1.8478375673294067, 'learning_rate': 4.7e-05, 'epoch': 0.11}
{'loss': 0.7946, 'grad_norm': 1.8938946723937988, 'learning_rate': 4.6500000000000005e-05, 'epoch': 0.12}
{'loss': 0.7617, 'grad_norm': 1.8960566520690918, 'learning_rate': 4.600000000000001e-05, 'epoch': 0.14}
{'loss': 0.7279, 'grad_norm': 1.863745927810669, 'learning_rate': 4.55e-05, 'epoch': 0.16}
{'loss': 0.747, 'grad_norm': 1.9353132247924805, 'learning_rate': 4.5e-05, 'epoch': 0.18}


  0%|          | 0/15 [00:00<?, ?it/s]

...

{'eval_loss': 1.062567949295044, 'eval_runtime': 32.3546, 'eval_samples_per_second': 3.678, 'eval_steps_per_second': 0.464, 'epoch': 1.59}
{'loss': 0.6099, 'grad_norm': 1.8475769758224487, 'learning_rate': 4.5e-06, 'epoch': 1.61}
{'loss': 0.5928, 'grad_norm': 1.70984947681427, 'learning_rate': 4.000000000000001e-06, 'epoch': 1.63}
{'loss': 0.5925, 'grad_norm': 1.947863221168518, 'learning_rate': 3.5000000000000004e-06, 'epoch': 1.65}
{'loss': 0.5887, 'grad_norm': 1.7866694927215576, 'learning_rate': 3e-06, 'epoch': 1.66}
{'loss': 0.6494, 'grad_norm': 1.953308343887329, 'learning_rate': 2.5e-06, 'epoch': 1.68}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.0621213912963867, 'eval_runtime': 31.5744, 'eval_samples_per_second': 3.769, 'eval_steps_per_second': 0.475, 'epoch': 1.68}
{'loss': 0.6383, 'grad_norm': 1.843813180923462, 'learning_rate': 2.0000000000000003e-06, 'epoch': 1.7}
{'loss': 0.5964, 'grad_norm': 1.7836089134216309, 'learning_rate': 1.5e-06, 'epoch': 1.72}
{'loss': 0.5975, 'grad_norm': 1.7674951553344727, 'learning_rate': 1.0000000000000002e-06, 'epoch': 1.73}
{'loss': 0.5925, 'grad_norm': 1.5956822633743286, 'learning_rate': 5.000000000000001e-07, 'epoch': 1.75}
{'loss': 0.5001, 'grad_norm': 1.5640424489974976, 'learning_rate': 0.0, 'epoch': 1.77}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.0615426301956177, 'eval_runtime': 30.3185, 'eval_samples_per_second': 3.925, 'eval_steps_per_second': 0.495, 'epoch': 1.77}
{'train_runtime': 4252.3493, 'train_samples_per_second': 0.941, 'train_steps_per_second': 0.235, 'train_loss': 0.6903949475288391, 'epoch': 1.77}


In [21]:
trainer.push_to_hub(tags=finetune_tags)

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Shern0507/SmolLM2-FT-MyDataset/commit/3c7bd19fe74695c6689326c7a4bbe4ebc857a474', commit_message='End of training', commit_description='', oid='3c7bd19fe74695c6689326c7a4bbe4ebc857a474', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Shern0507/SmolLM2-FT-MyDataset', endpoint='https://huggingface.co', repo_type='model', repo_id='Shern0507/SmolLM2-FT-MyDataset'), pr_revision=None, pr_num=None)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [22]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.